In [ ]:
#Per importare librerie e dati
import cudf as cf

eurgbp = cf.read_csv('/home/letizia/Scrivania/python_dir/eurgbp-m1-bid-2015-01-01-2025-02-07T20:43.csv', index_col=0)

In [ ]:
def backtest_donchian_sl_tp(df, window=20, stop_loss_pct=0.01, take_profit_pct=0.02, leva=10):
    # Calcolo canali Donchian (shiftati per evitare look-ahead bias)

    df = df * leva

    upper = df['high'].rolling(window=window, min_periods=window).max().shift(1)
    lower = df['low'].rolling(window=window, min_periods=window).min().shift(1)

    # Segnale base
    df['signal'] = 0
    df.loc[df['close'] > upper, 'signal'] = 1
    df.loc[df['close'] < lower, 'signal'] = -1
    df['signal'] = df['signal'].ffill()

    # Rendimenti
    df['ret'] = df['close'].pct_change().fillna(0)
   
   

    # Prezzo di entrata ffill
    df['entry_price'] = df['close'].where(df['signal'].diff() != 0).ffill()

    # Stop loss e take profit
    stop_cond = ((df['signal'] == 1) & (df['close'] <= df['entry_price'] * (1 - (stop_loss_pct/100) ))) | \
                ((df['signal'] == -1) & (df['close'] >= df['entry_price'] * (1 + (stop_loss_pct/100))))
    
    tp_cond = ((df['signal'] == 1) & (df['close'] >= df['entry_price'] * (1 + (take_profit_pct/100 )))) | \
              ((df['signal'] == -1) & (df['close'] <= df['entry_price'] * (1 - (take_profit_pct/100) )))
            
    # Esco su stop o take profit
    df['exit'] = (stop_cond | tp_cond)
    
    # Implementiamo lo Stop Loss:
    # Se una posizione long e il prezzo scende sotto l'entry_price del stop loss,
    # o se una posizione short e il prezzo sale sopra l'entry_price del stop loss,
    # la posizione viene chiusa.
    df.loc[df['exit'], 'signal'] = 0
    df['signal'] = df['signal'].ffill()

    # Strategia
    df['strategy_ret'] = df['signal'].shift(1).fillna(0) * df['ret']
    df['cum_return'] = (1 + df['strategy_ret']).cumprod()

    return df['cum_return']

In [ ]:
test = backtest_donchian_sl_tp(eurgbp, window=20, stop_loss_pct=0.01, take_profit_pct=0.02)
print(test) 

Questo frammento di codice crea una nuova colonna, chiamata `entry_price`, basandosi sulla colonna `close`. Il metodo `where` mantiene il valore di `close` solo quando la condizione `df['signal'].diff() != 0` è vera, mentre riempie con `NaN` quando è falsa.

Il risultato di questa operazione viene successivamente propagato in avanti con la funzione `ffill()`, che sostituisce i valori `NaN` con l’ultimo valore valido incontrato. In pratica, ogni volta che la differenza di `signal` è diversa da zero (indicando un possibile cambio di segnale), la colonna `entry_price` viene aggiornata con il prezzo di chiusura corrente; altrimenti, il prezzo precedente rimane immutato.

Questa porzione di codice gestisce l’uscita dalle posizioni in base alla logica di stop loss e take profit. La variabile **stop_cond** verifica quando il prezzo raggiunge un livello sfavorevole (sotto lo stop per i long e sopra lo stop per gli short), mentre la variabile **tp_cond** effettua il controllo opposto per il raggiungimento del target di profitto.

In seguito, **df['exit']** consolida entrambe le condizioni e, se una di queste è soddisfatta, segnala che la posizione deve essere chiusa. Infine, **mask** imposta il segnale a zero quando si verifica l’uscita, e la funzione **ffill()** propaga in avanti i valori del segnale per evitare buchi nei dati tra una chiusura e l’apertura successiva.

In [ ]:
test = backtest_donchian_sl_tp(eurgbp, window=30, stop_loss_pct=0.01, take_profit_pct=0.02)
print(test.tail())

### Optimization function

In [ ]:
import cupy as cp

combinations = cp.arange(13000, 20160, 30)  # Combinazioni di finestre temporali (start from 2 to avoid window=0)

results = []
for window in combinations:
    results.append(backtest_donchian(eurgbp, window=int(window)).iloc[-1])
    print(f'Finestra temporale: {window}, Rendimento finale: {results[-1]}')

best_window = combinations[cp.argmax(cp.array(results))]
print(f'Migliore finestra temporale: {best_window}')

In [ ]:
test = backtest_donchian_sl_tp(eurgbp, window=30, stop_loss_pct=0.001, take_profit_pct=1, leva=10)
test.to_pandas().plot(figsize=(14, 7), title='Backtest Donchian Channel Strategy')
test.loc[-1]

### WORK IN PROGRESS
C'è qualcosa che non va con lo stop loss, non si triggera. Stessa cosa per il take profit. Cercare come implementarlo in pandas.